## Carregando as Bibliotecas

In [1]:
#!pip install pywin32
import win32com.client as client
import pandas as pd
import numpy as np
import smtplib
import sys

## Carregando a tabela

In [2]:
#Realizando a leitura das abas das tabelas
tabela = pd.read_excel('Case Técnico - Data Operation - Service Delivery Database.xlsx', engine="openpyxl")
tabela2 = pd.read_excel('Case Técnico - Data Operation - Service Delivery Database.xlsx', sheet_name='CONTATOS', engine="openpyxl")  

#Analisando dados nulos

#tabela.info()

#Identificando os IDs das Empresas
tabela = tabela.assign(dado_id = tabela['EMPRESA_ID']) #gerando nova coluna "dado_id" com os mesmos dados da coluna EMPRESA_ID
#tabela = tabela.assign(capital_ = tabela['EMPRESA_ID']) #gerando nova coluna "dado_id" com os mesmos dados da coluna EMPRESA_ID
tabela['dado_id'] = tabela['dado_id'].str.replace(r'\D','') #removendo os dados e caracteres não numéricos da coluna "dado_id"
#display(tabela['dado_id'])
#display(tabela['EMPRESA_ID'])

C:\Users\khomp\AppData\Local\Temp\ipykernel_20072\2279596796.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tabela['dado_id'] = tabela['dado_id'].str.replace(r'\D','') #removendo os dados e caracteres não numéricos da coluna "dado_id"


## Ajustando os dados

In [3]:
tabela3 = tabela

#Ajustando os dados (quando nulo, alterar pelo texto + dado_id)
tabela['NOME FANTASIA'] = tabela['NOME FANTASIA'].fillna('NOME FANTASIA ' + tabela['dado_id']) 
tabela['ENDERECO COMPLEMENTO'] = tabela['ENDERECO COMPLEMENTO'].fillna('COMPLEMENTO ' + tabela['dado_id']) 

#Convertendo para numerico
tabela['CAPITAL SOCIAL'] = tabela['CAPITAL SOCIAL'].str.replace(r'\D','') #removendo texto não numérico
tabela['CAPITAL SOCIAL'] = pd.to_numeric(tabela['CAPITAL SOCIAL']) #convertendo para numerico

tabela['FATURAMENTO'] = tabela['FATURAMENTO'].str.replace(r'\D','')
tabela['FATURAMENTO'] = pd.to_numeric(tabela['FATURAMENTO'])

tabela['FUNCIONARIOS'] = tabela['FUNCIONARIOS'].str.replace(r'\D','')
tabela['FUNCIONARIOS'] = pd.to_numeric(tabela['FUNCIONARIOS'])

C:\Users\khomp\AppData\Local\Temp\ipykernel_20072\1972900325.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  tabela['CAPITAL SOCIAL'] = tabela['CAPITAL SOCIAL'].str.replace(r'\D','') #removendo texto não numérico
C:\Users\khomp\AppData\Local\Temp\ipykernel_20072\1972900325.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  tabela['FATURAMENTO'] = tabela['FATURAMENTO'].str.replace(r'\D','')
C:\Users\khomp\AppData\Local\Temp\ipykernel_20072\1972900325.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  tabela['FUNCIONARIOS'] = tabela['FUNCIONARIOS'].str.replace(r'\D','')


## Substituindo valores nulos pela média

In [5]:
tabela['CAPITAL SOCIAL'].fillna(value=tabela['CAPITAL SOCIAL'].median(),inplace=True)
tabela['FATURAMENTO'].fillna(value=tabela['FATURAMENTO'].median(),inplace=True)
tabela['FUNCIONARIOS'].fillna(value=tabela['FUNCIONARIOS'].median(),inplace=True)
dados_nulos = tabela.isnull().sum() 
print(dados_nulos)

EMPRESA_ID                 0
RAZAO SOCIAL               0
NOME FANTASIA              0
ENDERECO                   0
ENDERECO NUMERO            5
ENDERECO COMPLEMENTO       0
BAIRRO                     0
CIDADE                     0
ESTADO                     0
CEP                        0
TAMANHO                    0
CAPITAL SOCIAL             0
FATURAMENTO                0
FUNCIONARIOS               0
DATA DE ABERTURA           0
TIPO DE UNIDADE            0
SITUACAO                   0
ATIVIDADE PRINCIPAL        0
NATUREZA JURIDICA          0
TELEFONE 1                 0
TELEFONE 2               533
TELEFONE 3              1042
TELEFONE 4              1584
TELEFONE 5              2054
WEBSITE 1                755
WEBSITE 2               2401
WEBSITE 3               2959
WEBSITE 4               3213
WEBSITE 5               3355
dado_id                    0
dtype: int64


## Ajustando as datas

In [6]:
tabela['DATA DE ABERTURA'] = pd.to_datetime(tabela['DATA DE ABERTURA']) #altera para o padrão de data e hora 
tabela['DATA DE ABERTURA']  = tabela['DATA DE ABERTURA'] .dt.strftime('%m/%d/%Y') #padroniza para data mes/dia/ano

C:\Users\khomp\AppData\Local\Temp\ipykernel_20072\4024629839.py:1: UserWarning: Parsing '25/02/2002' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  tabela['DATA DE ABERTURA'] = pd.to_datetime(tabela['DATA DE ABERTURA']) #altera para o padrão de data e hora
C:\Users\khomp\AppData\Local\Temp\ipykernel_20072\4024629839.py:1: UserWarning: Parsing '30/09/2004' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  tabela['DATA DE ABERTURA'] = pd.to_datetime(tabela['DATA DE ABERTURA']) #altera para o padrão de data e hora
C:\Users\khomp\AppData\Local\Temp\ipykernel_20072\4024629839.py:1: UserWarning: Parsing '29/03/2007' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  tabela['DATA DE ABERTURA'] = pd.to_datetime(tabela['DATA DE ABERTURA']) #altera para o padrão de data e hora
C:\Users\khomp\AppData\Local\Temp\ipykernel_20072\4024629839.py:1: 

##  Ajustando os dados por categoria

In [7]:
condition = [(tabela['CAPITAL SOCIAL'] > 0) & (tabela['CAPITAL SOCIAL'] <= 5000000),                # R$ 0 - R$ 50 K
             (tabela['CAPITAL SOCIAL'] > 5000000) & (tabela['CAPITAL SOCIAL'] <= 25000000),         # R$ 51K - R$ 250 K
             (tabela['CAPITAL SOCIAL'] > 25000000) & (tabela['CAPITAL SOCIAL'] <= 50000000),        # R$ 251 - R$ 500 K
             (tabela['CAPITAL SOCIAL'] > 50000000) & (tabela['CAPITAL SOCIAL'] <= 250000000),       # R$ 501 - R$ 2,5 M
             (tabela['CAPITAL SOCIAL'] > 250000000) & (tabela['CAPITAL SOCIAL'] <= 1000000000),     # R$ 2,5 M - R$ 10 M
             (tabela['CAPITAL SOCIAL'] > 1000000000) & (tabela['CAPITAL SOCIAL'] <= 5000000000),    # R$ 10 M - R$ 50 M
             (tabela['CAPITAL SOCIAL'] > 5000000000) & (tabela['CAPITAL SOCIAL'] <= 10000000000),   # R$ 50 M - R$ 100 M
             (tabela['CAPITAL SOCIAL'] > 10000000000) & (tabela['CAPITAL SOCIAL'] <= 25000000000),  # R$ 100 M - R$ 250 M
             (tabela['CAPITAL SOCIAL'] > 25000000000) & (tabela['CAPITAL SOCIAL'] <= 50000000000),  # R$ 250 M - R$ 500 M
             (tabela['CAPITAL SOCIAL'] > 50000000000) & (tabela['CAPITAL SOCIAL'] <= 100000000000), # R$ 500 M - R$ 1 B
             (tabela['CAPITAL SOCIAL'] <= 100000000000)                                             # + R$ 1 B
]

choices = ['R$ 0 - R$ 50 K',
           'R$ 51K - R$ 250 K',
           'R$ 251 - R$ 500 K',
           'R$ 501 - R$ 2,5 M',
           'R$ 2,5 M - R$ 10 M',
           'R$ 10 M - R$ 50 M',
           'R$ 50 M - R$ 100 M',
           'R$ 100 M - R$ 250 M',
           'R$ 250 M - R$ 500 M',  
           'R$ 500 M - R$ 1 B',
           '+ R$ 1 B']
tabela['CAPITAL SOCIAL'] = np.select(condition, choices, default='null')


condition = [(tabela['FUNCIONARIOS'] > 0) & (tabela['FUNCIONARIOS'] <= 3),                # R$ 0 - R$ 50 K
             (tabela['FUNCIONARIOS'] > 3) & (tabela['FUNCIONARIOS'] <= 10),         # R$ 51K - R$ 250 K
             (tabela['FUNCIONARIOS'] > 10) & (tabela['FUNCIONARIOS'] <= 50),        # R$ 251 - R$ 500 K
             (tabela['FUNCIONARIOS'] > 50) & (tabela['FUNCIONARIOS'] <= 200),       # R$ 501 - R$ 2,5 M
             (tabela['FUNCIONARIOS'] > 200) & (tabela['FUNCIONARIOS'] <= 500),     # R$ 2,5 M - R$ 10 M
             (tabela['FUNCIONARIOS'] > 500) & (tabela['FUNCIONARIOS'] <= 500),    # R$ 10 M - R$ 50 M
             (tabela['FUNCIONARIOS'] > 1000) & (tabela['FUNCIONARIOS'] <= 5000),   # R$ 50 M - R$ 100 M
             (tabela['FUNCIONARIOS'] > 5000) & (tabela['FUNCIONARIOS'] <= 10000)  # R$ 100 M - R$ 250 M                                             # + R$ 1 B
]

choices = ['0 - 3',
           '4 - 10',
           '11 - 50',
           '51 - 200',
           '201 - 500',
           '501 - 1000',
           '1001 - 5000',
           '5001 - 1000']

tabela['FUNCIONARIOS'] = np.select(condition, choices, default='null')

## Exportando os dados

In [8]:
tabela = tabela.drop(["dado_id"], axis=1) #remove coluna de apoio "dado_id"
tabela.to_csv("Tabela_final.csv")

## Questões

In [9]:
#Quantidade total de empresas
quantidade_empresas = tabela['EMPRESA_ID']
quantidade_empresas = quantidade_empresas.count()
print("A quantidade de empresas cadastradas é:", quantidade_empresas)

#Quantos contatos existem na base?
quantidade_contatos = tabela2['EMPRESA_ID']
quantidade_contatos = quantidade_contatos.count()
print("A quantidade de contatos existens na base é:", quantidade_contatos)

#Quantos websites únicos existem na base?
web1 = tabela['WEBSITE 1']
web1 = web1.count()
web2 = tabela['WEBSITE 2']
web2 = web2.count()
web3 = tabela['WEBSITE 3']
web3 = web3.count()
web4 = tabela['WEBSITE 4']
web4 = web4.count()
web5 = tabela['WEBSITE 5']
web5 = web5.count()
web = [web1, web2, web3, web4, web5]
quantidade_web = sum(web)
print("A quantidade de websites únicos existem na base é:",quantidade_web)

#Quantos e-mail existem na base?
quantidade_email = tabela2['E-MAIL ']
quantidade_email = quantidade_email.count()
print("A quantidade de e-mails existentes é:", quantidade_email)

A quantidade de empresas cadastradas é: 3694
A quantidade de contatos existens na base é: 16576
A quantidade de websites únicos existem na base é: 5787
A quantidade de e-mails existentes é: 8619
